# Data pool の操作 

## 1. スケールアウトテーブルの作成

In [1]:
USE [DataPool];

-- オブジェクトの初期化
IF EXISTS (SELECT * FROM sys.external_tables WHERE name = 'clickstream_datamart')
BEGIN
	DROP EXTERNAL TABLE clickstream_datamart
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.0168294

**1. Data Pool に、スケールアウトデータマート用のテーブルを作成**  
本クエリにより、Data Pool 内の各 SQL Server にテーブルが作成される  
(テーブル作成が Data Pool の各 SQL Server に伝搬されるまで多少のタイムラグがある)

作成されたテーブルは列ストアインデックスが自動的に作成された状態となっている

In [2]:
USE [DataPool];

-- テーブルの作成
CREATE EXTERNAL TABLE [clickstream_datamart]
(
	wcs_click_date_sk BIGINT , 
	wcs_click_time_sk BIGINT , 
	wcs_sales_sk BIGINT , 
	wcs_item_sk BIGINT , 
	wcs_web_page_sk BIGINT , 
	wcs_user_sk BIGINT
) 
WITH
( 
	DATA_SOURCE = SqlDataPool,	
	DISTRIBUTION = ROUND_ROBIN
);
GO
WAITFOR DELAY '00:00:10'

Commands completed successfully.

Total execution time: 00:00:07.9955798

Commands completed successfully.

Total execution time: 00:00:10.1368189

**2. リンクサーバー経由で確認**  
Master Instance に設定したリンクサーバー経由で、Data Pool 内の SQL Server の情報を確認

In [3]:
USE [DataPool];

SELECT
	(SELECT name from [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
UNION
SELECT
	(SELECT name from [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
GO

(2 rows affected)

Total execution time: 00:00:02.2784387

server_name,name,name,type_desc,count
data-0-0,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,0
data-0-1,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,0


![Linked Server](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/03.Data%20Store/02.Data%20Pool/Data%20Pool%20Database%20object.png)

## 2. スケールアウトデータマートにデータの投入

**1. データ投入**  
INSERTO SELECT でスケールアウトデータマートのテーブルを指定することで、データマートにデータを投入することができる。  

In [4]:
USE [DataPool];
INSERT INTO clickstream_datamart
SELECT * FROM StoragePool..StoragePoolTBL
-- SELECT * FROM StoragePool..StoragePoolADLTBL
-- SELECT * FROM StoragePool..StoragePoolS3TBL

(1497 rows affected)

Total execution time: 00:00:05.6709408

**2. 投入されたデータの確認**  
リンクサーバー経由で投入されたデータの状況を確認

In [5]:
USE [DataPool];

SELECT
	(SELECT name from [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-0.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
UNION
SELECT
	(SELECT name from [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.servers WHERE server_id = 0) AS server_name
	, o.name
	, i.name
	, i.type_desc
	, (SELECT COUNT(*) FROM [DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.dbo.clickstream_datamart) AS count
FROM
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.indexes AS i
	LEFT JOIN 
	[DATA-0-1.DATA-0-SVC.MSSQL-CLUSTER.SVC.CLUSTER.LOCAL].DataPool.sys.objects AS o
	ON i.object_id = o.object_id
WHERE
	o.name = 'clickstream_datamart'
GO

(2 rows affected)

Total execution time: 00:00:00.1899467

server_name,name,name,type_desc,count
data-0-0,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,499
data-0-1,clickstream_datamart,cci,CLUSTERED COLUMNSTORE,998


**3. スケールアウトデータマーのデータ検索**  
スケールアウトデータマートに投入されているデータは、Master Instance からデータを取得することができる。

In [6]:
USE [DataPool];

SELECT TOP 25 * FROM clickstream_datamart

(25 rows affected)

Total execution time: 00:00:03.5009633

wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
38569,4250,NULL,7840,18,NULL
38569,85106,NULL,11130,18,NULL
38569,52655,NULL,3716,18,NULL
38569,70934,NULL,13243,18,NULL
38569,40166,NULL,5389,18,NULL
38570,73271,NULL,3331,18,NULL
38570,24651,NULL,10049,18,NULL
38570,23805,NULL,921,18,NULL
38570,66458,NULL,4407,18,NULL
38570,65912,NULL,11494,18,NULL


![Scale Out Data mart](https://raw.githubusercontent.com/MasayukiOzawa/decode-2019-demo/master/Images/03.Data%20Store/02.Data%20Pool/Scale-out%20data%20mart%20select.png)